In [9]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [11]:
subject = 'Acre - Consumo de Cimento (t)'
start_index = 60
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [12]:
data = pd.read_csv('2003_mo_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Consumo de Cimento (t)
0,2003-1,0.773065,0.651726,8.722906,0.691320,7.285707e+06,413409.255032,9.318417,6.587220e+06,0.643805,...,1.921789e+10,8.202881e+10,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,3.260
1,2003-2,0.773454,0.651847,8.718028,0.691617,7.294881e+06,413607.843560,9.319990,6.590338e+06,0.644270,...,1.921789e+10,8.222716e+10,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,2.191
2,2003-3,0.773843,0.651968,8.713149,0.691914,7.304055e+06,413806.432089,9.321563,6.593455e+06,0.644734,...,1.921789e+10,8.234287e+10,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,2.154
3,2003-4,0.774233,0.652088,8.708271,0.692212,7.313229e+06,414005.020618,9.323136,6.596572e+06,0.645199,...,1.921789e+10,8.235526e+10,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,2.643
4,2003-5,0.774622,0.652209,8.703393,0.692509,7.322403e+06,414203.609146,9.324709,6.599689e+06,0.645663,...,1.921843e+10,8.245980e+10,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,2.546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2022-9,NaN,0.661031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.032
237,2022-10,NaN,0.661074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.573
238,2022-11,NaN,0.661117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.493
239,2022-12,NaN,0.661160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.493


In [13]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.415882,-2.356162,-0.871260,-2.233199,-1.827644,0.550935,-0.265790,-2.404401,-2.033439,1.874594,...,1.155957,-1.063593,-0.822933,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-2.372431,-2.318388,-0.874207,-2.196498,-1.807633,0.567407,-0.242610,-2.361200,-2.005361,1.869312,...,1.155957,-1.063593,-0.818174,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-2.328980,-2.280615,-0.877154,-2.159796,-1.787622,0.583879,-0.219431,-2.317998,-1.977284,1.866230,...,1.155957,-1.063593,-0.815398,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-2.285529,-2.242841,-0.880101,-2.123094,-1.767611,0.600352,-0.196252,-2.274796,-1.949206,1.865900,...,1.155957,-1.063593,-0.815101,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-2.242078,-2.205067,-0.883048,-2.086392,-1.747600,0.616824,-0.173072,-2.231595,-1.921128,1.856658,...,1.151943,-1.063566,-0.812592,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0.848464,0.963521,1.086583,1.093639,0.845955,-0.984945,-1.492171,0.341015,1.238509,-0.624727,...,3.115817,1.690001,0.667867,-0.689886,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,0.826218,0.932813,1.079514,1.075237,0.829861,-0.967308,-1.480041,0.326774,1.225428,-0.548742,...,3.295619,1.717926,0.640959,-0.441954,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,0.803973,0.902105,1.072445,1.056835,0.813767,-0.949672,-1.467910,0.312533,1.212348,-0.439805,...,3.486686,1.748758,0.615514,-0.132782,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,0.781727,0.871397,1.065376,1.038433,0.797673,-0.932035,-1.455780,0.298292,1.199267,-0.327203,...,3.692016,1.780444,0.593546,0.084061,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [14]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0       2.191
1       2.154
2       2.643
3       2.546
4       3.040
        ...  
236    18.573
237    15.493
238    15.493
239       NaN
240       NaN
Name: Acre - Consumo de Cimento (t), Length: 241, dtype: float64

In [15]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
60,-0.373751,-0.468272,-0.997062,-0.464067,-0.803741,1.287584,1.298329,-0.503624,-0.570208,-0.253335,...,-0.660781,-0.907428,-1.023454,-0.537241,0.042292,0.488894,-0.717137,-0.601292,-0.315948,0.240813
61,-0.353360,-0.445613,-0.995666,-0.445007,-0.794643,1.300602,1.279278,-0.506236,-0.551494,-0.296485,...,-0.660781,-0.899853,-1.024085,-0.479316,0.011943,-0.072763,-0.714911,-0.580436,-0.286095,0.246566
62,-0.332969,-0.422953,-0.994271,-0.425946,-0.785545,1.313621,1.260227,-0.508848,-0.532780,-0.338941,...,-0.660781,-0.892039,-1.024307,-0.466312,-0.019339,-0.316940,-0.694168,-0.559580,-0.256243,0.315327
63,-0.312579,-0.400293,-0.992875,-0.406886,-0.776447,1.326639,1.241177,-0.511460,-0.514066,-0.380655,...,-0.660781,-0.883981,-1.024104,-0.438778,-0.059264,0.180777,-0.700209,-0.538723,-0.226390,0.356189
64,-0.292188,-0.377633,-0.991479,-0.387826,-0.767349,1.339658,1.222126,-0.514072,-0.495353,-0.421583,...,-0.660781,-0.875677,-1.023461,-0.457839,-0.104671,-0.212484,-0.660717,-0.517867,-0.196537,0.524165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.369760,1.036081,1.310355,1.290080,1.098918,-1.469526,-1.416170,0.668474,1.230573,-0.685481,...,-0.660781,1.272681,1.521503,-0.742853,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806
164,1.363312,1.010393,1.299668,1.287297,1.100953,-1.469505,-1.430567,0.666551,1.227875,-0.757697,...,-0.660781,1.288866,1.512161,-0.787367,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523
165,1.356865,0.984705,1.288982,1.284515,1.102988,-1.469485,-1.444964,0.664628,1.225176,-0.844729,...,-0.660781,1.304998,1.500026,-0.757940,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175
166,1.350417,0.959017,1.278295,1.281733,1.105023,-1.469464,-1.459361,0.662705,1.222477,-0.914820,...,-0.660781,1.332157,1.486995,-0.917469,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146


In [16]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

60      8.675000
61      6.078000
62      8.368000
63     11.648000
64     10.911000
         ...    
163    10.885543
164    10.765657
165    10.645771
166    10.525886
167    10.406000
Name: Acre - Consumo de Cimento (t), Length: 108, dtype: float64

In [17]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
168,1.337522,0.907641,1.256922,1.276169,1.109094,-1.469423,-1.488155,0.658858,1.217079,-1.028195,...,-0.660781,1.364625,1.460212,-1.032458,-0.585256,-0.505289,-1.202612,1.035545,0.707473,-1.446117
169,1.326260,0.912772,1.252043,1.283418,1.106798,-1.455041,-1.495061,0.654927,1.229249,-1.102985,...,-0.660781,1.371257,1.445627,-1.077854,-0.581056,-0.373612,-1.139987,1.046449,0.692399,-1.526274
170,1.314997,0.917903,1.247165,1.290668,1.104502,-1.440660,-1.501968,0.650996,1.241419,-1.168851,...,-0.660781,1.375328,1.430008,-1.036724,-0.574769,-0.291560,-1.053534,1.057352,0.677324,-1.610341
171,1.303734,0.923034,1.242287,1.297918,1.102206,-1.426278,-1.508874,0.647064,1.253589,-1.188592,...,-0.660781,1.381257,1.413730,-1.013400,-0.567021,-0.450118,-0.932974,1.068256,0.662249,-1.601301
172,1.292472,0.928165,1.237408,1.305168,1.099910,-1.411896,-1.515781,0.643133,1.265759,-1.180922,...,-0.660781,1.364882,1.396980,-1.059228,-0.557842,-0.909054,-0.763756,1.079159,0.647175,-1.618284
173,1.281209,0.933297,1.232530,1.312418,1.097614,-1.397514,-1.522688,0.639202,1.277928,-1.124581,...,-0.582593,1.369812,1.379116,-1.170032,-0.548012,0.387442,-0.644652,1.090063,0.632100,-1.602348
174,1.269947,0.938428,1.227652,1.319668,1.095318,-1.383133,-1.529594,0.635270,1.290098,-1.064934,...,-0.474908,1.376543,1.360141,-1.115621,-0.537244,-0.937589,-0.541601,1.100966,0.617026,-1.686659
175,1.258684,0.943559,1.222773,1.326918,1.093022,-1.368751,-1.536501,0.631339,1.302268,-1.111858,...,-0.364364,1.384496,1.341361,-1.182159,-0.524149,-1.212466,-0.565747,1.111870,0.601951,-1.688184
176,1.247421,0.948690,1.217895,1.334168,1.090727,-1.354369,-1.543408,0.627408,1.314438,-1.162547,...,-0.250935,1.391358,1.323058,-1.270623,-0.519647,-0.499856,-0.536256,1.122773,0.586877,-1.692173
177,1.236159,0.953821,1.213016,1.341418,1.088431,-1.339988,-1.550314,0.623476,1.326608,-1.165964,...,-0.134596,1.399740,1.303196,-1.321931,-0.510389,-0.120028,-0.544036,1.133677,0.571802,-1.690760


In [18]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

168     9.081
169     9.545
170    10.130
171    14.451
172    13.218
173    13.552
174    15.826
175    14.869
176    14.734
177    12.754
178    11.925
179    13.114
180     8.920
181     9.724
182     9.737
183    10.136
184    14.403
185    12.871
186    15.749
187    14.269
188    15.367
189    14.595
190     9.811
191    11.188
192    10.381
193     8.080
194    11.154
195    12.508
196    12.126
197    14.496
198    16.723
199    15.253
200    16.531
201    15.206
202    10.160
203    13.188
Name: Acre - Consumo de Cimento (t), dtype: float64

In [19]:
def validation_splitter(df, div_factor, add_factor):
    split_factor = len(df)//div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop_index.append(pos + add_factor)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop_index), df.iloc[positions_to_drop]

In [20]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 10, 60)
    target,target_val = validation_splitter(train_target, 10, 60)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [23]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [24]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1755839948, 1735766512, 2026919029, 177568464, 300237744, 3553417352, 272724036, 1320672778, 3918081930, 3104468012, 3515162554, 472816925, 1096044492, 2913337207, 1073587569, 3635991465, 1038067452, 3854772330, 472894281, 929393152, 4163134334, 4054495080, 1633787593, 3794668019, 1770108017]


Step: 0 ___________________________________________
val_loss: 0.6314786672592163
winner_seed: 1755839948


Step: 1 ___________________________________________
val_loss: 0.7139540910720825


Step: 2 ___________________________________________
val_loss: 0.8832440376281738


Step: 3 ___________________________________________
val_loss: 0.7206495404243469


Step: 4 ___________________________________________
val_loss: 0.8809696435928345


Step: 5 ___________________________________________
val_loss: 0.6196497082710266
winner_seed: 3553417352


Step: 6 ___________________________________________
val_loss: 0.5583799481391907
winner_seed: 272724036


Step: 7 ___________________________________________

In [25]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 0s 32ms/step - loss: 190.8968 - val_loss: 16.3203
Epoch 2/10000
4/4 [==============================] - 0s 6ms/step - loss: 16.5205 - val_loss: 98.0375
Epoch 3/10000
4/4 [==============================] - 0s 6ms/step - loss: 18.8110 - val_loss: 3.2650
Epoch 4/10000
4/4 [==============================] - 0s 5ms/step - loss: 9.2011 - val_loss: 6.6293
Epoch 5/10000
4/4 [==============================] - 0s 5ms/step - loss: 13.4739 - val_loss: 8.7512
Epoch 6/10000
4/4 [==============================] - 0s 5ms/step - loss: 11.0747 - val_loss: 2.4103
Epoch 7/10000
4/4 [==============================] - 0s 6ms/step - loss: 8.3873 - val_loss: 5.3172
Epoch 8/10000
4/4 [==============================] - 0s 6ms/step - loss: 10.4928 - val_loss: 40.0379
Epoch 9/10000
4/4 [==============================] - 0s 6ms/step - loss: 14.9203 - val_loss: 3.5938
Epoch 10/10000
4/4 [==============================] - 0s 6ms/step - loss: 10.5419 - val_loss: 3.5

Epoch 83/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.2794 - val_loss: 1.4528
Epoch 84/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.9619 - val_loss: 2.2174
Epoch 85/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.1239 - val_loss: 4.5550
Epoch 86/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.3953 - val_loss: 3.4807
Epoch 87/10000
4/4 [==============================] - 0s 5ms/step - loss: 10.8684 - val_loss: 1.7588
Epoch 88/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.9150 - val_loss: 2.3519
Epoch 89/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.8270 - val_loss: 1.0105
Epoch 90/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.8024 - val_loss: 8.8398
Epoch 91/10000
4/4 [==============================] - 0s 6ms/step - loss: 10.2617 - val_loss: 2.5225
Epoch 92/10000
4/4 [==============================] - 0s 6ms/step - loss: 7.6471 - val_loss: 1.667

4/4 [==============================] - 0s 5ms/step - loss: 8.3116 - val_loss: 3.5234
Epoch 165/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.6433 - val_loss: 0.9366
Epoch 166/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.7168 - val_loss: 1.9896
Epoch 167/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.8943 - val_loss: 1.2330
Epoch 168/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.0496 - val_loss: 3.6552
Epoch 169/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.3016 - val_loss: 1.5595
Epoch 170/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.0839 - val_loss: 1.2169
Epoch 171/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.6459 - val_loss: 1.3655
Epoch 172/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.5509 - val_loss: 3.0084
Epoch 173/10000
4/4 [==============================] - 0s 6ms/step - loss: 9.0543 - val_loss: 4.2219
Epoch 

4/4 [==============================] - 0s 5ms/step - loss: 6.4949 - val_loss: 1.2326
Epoch 246/10000
4/4 [==============================] - 0s 6ms/step - loss: 7.2132 - val_loss: 1.4678
Epoch 247/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.8548 - val_loss: 2.7391
Epoch 248/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.5372 - val_loss: 2.8359
Epoch 249/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.8284 - val_loss: 4.1228
Epoch 250/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.3102 - val_loss: 1.2220
Epoch 251/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.7849 - val_loss: 1.1057
Epoch 252/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.9909 - val_loss: 1.1665
Epoch 253/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.1214 - val_loss: 1.0920
Epoch 254/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.6541 - val_loss: 1.6727
Epoch 

4/4 [==============================] - 0s 6ms/step - loss: 11.6635 - val_loss: 3.0730
Epoch 327/10000
4/4 [==============================] - 0s 5ms/step - loss: 10.0420 - val_loss: 1.5156
Epoch 328/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.1833 - val_loss: 6.9862
Epoch 329/10000
4/4 [==============================] - 0s 5ms/step - loss: 13.1468 - val_loss: 1.5883
Epoch 330/10000
4/4 [==============================] - 0s 6ms/step - loss: 7.3728 - val_loss: 2.6711
Epoch 331/10000
4/4 [==============================] - 0s 5ms/step - loss: 11.2108 - val_loss: 1.7023
Epoch 332/10000
4/4 [==============================] - 0s 6ms/step - loss: 7.5633 - val_loss: 12.5288
Epoch 333/10000
4/4 [==============================] - 0s 5ms/step - loss: 10.9169 - val_loss: 3.3135
Epoch 334/10000
4/4 [==============================] - 0s 5ms/step - loss: 8.9876 - val_loss: 2.2337
Epoch 335/10000
4/4 [==============================] - 0s 5ms/step - loss: 9.6404 - val_loss: 1.2414


...

4/4 [==============================] - 0s 5ms/step - loss: 5.9889 - val_loss: 3.5287
Epoch 1295/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.7633 - val_loss: 2.6906
Epoch 1296/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.8670 - val_loss: 3.7700
Epoch 1297/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.7244 - val_loss: 1.6910
Epoch 1298/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.4249 - val_loss: 8.7824
Epoch 1299/10000
4/4 [==============================] - 0s 6ms/step - loss: 8.2977 - val_loss: 9.6602
Epoch 1300/10000
4/4 [==============================] - 0s 5ms/step - loss: 12.5190 - val_loss: 1.0142
Epoch 1301/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.4513 - val_loss: 1.3013
Epoch 1302/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.9106 - val_loss: 16.9101
Epoch 1303/10000
4/4 [==============================] - 0s 5ms/step - loss: 11.6992 - val_loss: 3

4/4 [==============================] - 0s 5ms/step - loss: 6.1217 - val_loss: 2.0157
Epoch 1375/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.3032 - val_loss: 3.9017
Epoch 1376/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.5369 - val_loss: 6.4438
Epoch 1377/10000
4/4 [==============================] - 0s 6ms/step - loss: 5.9233 - val_loss: 6.1087
Epoch 1378/10000
4/4 [==============================] - 0s 6ms/step - loss: 5.7607 - val_loss: 2.6998
Epoch 1379/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.3001 - val_loss: 0.8221
Epoch 1380/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.1333 - val_loss: 1.6738
Epoch 1381/10000
4/4 [==============================] - 0s 5ms/step - loss: 4.5163 - val_loss: 1.5957
Epoch 1382/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.8742 - val_loss: 3.4323
Epoch 1383/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.7593 - val_loss: 3.05

4/4 [==============================] - 0s 6ms/step - loss: 5.7751 - val_loss: 4.9079
Epoch 1455/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.7975 - val_loss: 6.9581
Epoch 1456/10000
4/4 [==============================] - 0s 6ms/step - loss: 5.9242 - val_loss: 7.5716
Epoch 1457/10000
4/4 [==============================] - 0s 6ms/step - loss: 10.4684 - val_loss: 3.1672
Epoch 1458/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.1753 - val_loss: 7.5125
Epoch 1459/10000
4/4 [==============================] - 0s 6ms/step - loss: 8.1230 - val_loss: 2.6497
Epoch 1460/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.4762 - val_loss: 3.0560
Epoch 1461/10000
4/4 [==============================] - 0s 6ms/step - loss: 5.7965 - val_loss: 2.9477
Epoch 1462/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.6989 - val_loss: 3.1849
Epoch 1463/10000
4/4 [==============================] - 0s 6ms/step - loss: 5.3420 - val_loss: 3.3

4/4 [==============================] - 0s 5ms/step - loss: 8.3474 - val_loss: 1.6142
Epoch 1535/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.3655 - val_loss: 1.6334
Epoch 1536/10000
4/4 [==============================] - 0s 5ms/step - loss: 4.7587 - val_loss: 1.2221
Epoch 1537/10000
4/4 [==============================] - 0s 5ms/step - loss: 4.9092 - val_loss: 3.4578
Epoch 1538/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.8922 - val_loss: 2.1826
Epoch 1539/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.6814 - val_loss: 2.9562
Epoch 1540/10000
4/4 [==============================] - 0s 5ms/step - loss: 7.0300 - val_loss: 1.7796
Epoch 1541/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.8650 - val_loss: 4.7404
Epoch 1542/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.6083 - val_loss: 4.2974
Epoch 1543/10000
4/4 [==============================] - 0s 6ms/step - loss: 8.6295 - val_loss: 2.15

4/4 [==============================] - 0s 6ms/step - loss: 7.3616 - val_loss: 8.2865
Epoch 1615/10000
4/4 [==============================] - 0s 5ms/step - loss: 8.3719 - val_loss: 6.0460
Epoch 1616/10000
4/4 [==============================] - 0s 5ms/step - loss: 10.8759 - val_loss: 2.0096
Epoch 1617/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.3488 - val_loss: 1.8589
Epoch 1618/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.4857 - val_loss: 2.2251
Epoch 1619/10000
4/4 [==============================] - 0s 6ms/step - loss: 4.8522 - val_loss: 4.3753
Epoch 1620/10000
4/4 [==============================] - 0s 6ms/step - loss: 6.2762 - val_loss: 2.2848
Epoch 1621/10000
4/4 [==============================] - 0s 6ms/step - loss: 5.7001 - val_loss: 1.7533
Epoch 1622/10000
4/4 [==============================] - 0s 5ms/step - loss: 6.2018 - val_loss: 1.7501
Epoch 1623/10000
4/4 [==============================] - 0s 5ms/step - loss: 5.4239 - val_loss: 4.2

In [26]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [27]:
errors, mae, mape = mae_mape_calculator(trained_model, test_input, test_target, 168)

1/1 [==============================] - 0s 48ms/step
Month-1 - Error: [[1.8069715]]
1/1 [==============================] - 0s 13ms/step
Month-2 - Error: [[1.1577368]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[0.34623337]]
1/1 [==============================] - 0s 13ms/step
Month-4 - Error: [[4.3336334]]
1/1 [==============================] - 0s 12ms/step
Month-5 - Error: [[3.7840052]]
1/1 [==============================] - 0s 12ms/step
Month-6 - Error: [[4.162528]]
1/1 [==============================] - 0s 12ms/step
Month-7 - Error: [[8.794777]]
1/1 [==============================] - 0s 12ms/step
Month-8 - Error: [[10.036367]]
1/1 [==============================] - 0s 13ms/step
Month-9 - Error: [[10.539999]]
1/1 [==============================] - 0s 13ms/step
Month-10 - Error: [[9.815354]]
1/1 [==============================] - 0s 13ms/step
Month-11 - Error: [[10.761404]]
1/1 [==============================] - 0s 13ms/step
Month-12 - Error: [[16.506886]]
1/1

In [28]:
display(mae)
display(mape)

13.958256

1.1069049

In [29]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [30]:
errors, mae, mape = year_mae_mape_calculator(trained_model, test_input, test_target, 168)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[77.774994]] - Target[153.199]| =  Error: [[75.42401]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[-57.451874]] - Target[146.76999999999998]| =  Error: [[204.22188]]
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[-60.42341]] - Target[155.80599999999995]| =  Error: [[216.2294]]


[array([[75.42401]], dtype=float32),
 array([[204.22188]], dtype=float32),
 array([[216.2294]], dtype=float32)]

165.29176

1.0905267